In [1]:
import os

In [2]:
%pwd

'/Users/ravina/Desktop/TextSummarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/ravina/Desktop/TextSummarizer'

In [5]:
#entity: return type of our function
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)  #this is not python class but dataclass, here you can define the veriables without using self keyword.
class DataTransformationConfig:
    root_dir:Path
    data_path:Path
    tokenizer_name:Path

In [6]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml,create_directories


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        

        create_directories([self.config.artifacts_root])



    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.get('data_transformation')
        if config is None:
            raise ValueError("Data Ingestion configuration is missing or invalid.")

        create_directories([config.get('root_dir')])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config 

In [12]:
import os
from textsummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk


In [13]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self,example_batch):
        input_encodings=self.tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings=self.tokenizer(example_batch['summary'],max_length=128,truncation=True)


        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        } 
    

    def convert(self):
        dataset_samsum=load_from_disk(self.config.data_path)
        dataset_samsum_pt=dataset_samsum.map(self.convert_examples_to_features,batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,'samsum_dataset'))

In [14]:
#pipeline
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
    
except Exception as e:
    raise e

[2024-02-25 13:47:15,003: INFO: common: yaml file:config/config.yaml loaded successfully]
[2024-02-25 13:47:15,006: INFO: common: yaml file:params.yaml loaded successfully]
[2024-02-25 13:47:15,008: INFO: common: created directory at:artifacts]
[2024-02-25 13:47:15,009: INFO: common: created directory at:artifacts/data_transformation]


tokenizer_config.json: 100%|██████████| 88.0/88.0 [00:00<00:00, 48.1kB/s]
config.json: 100%|██████████| 1.12k/1.12k [00:00<00:00, 1.06MB/s]
spiece.model: 100%|██████████| 1.91M/1.91M [00:00<00:00, 3.29MB/s]
special_tokens_map.json: 100%|██████████| 65.0/65.0 [00:00<00:00, 139kB/s]
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/Users/ravina/Desktop/TextSummarizer/textvenv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 158064.16 examples/s]
